In [1]:
import numpy as np
import cv2
from PIL import Image
import scipy.ndimage 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
import os
import re
import subprocess
from scipy.ndimage import median_filter

# Блок со всеми используемыми функциями

In [ ]:
"""Попробуем реализовать фильтр log + zero-crossing"""
def log_filter(image: np.ndarray) -> np.ndarray:
    """
    Log фильр, возвращает отфильтрованное изоражение
    Параметры:
        image (np.ndarray): изоражение, получаемое на вход для обработки, изображение должно быть в градациях серого!
    Обработка 2х граничных пикселов не будет проводиться.
    """
    kernel_size = 5
    kernel = np.array([0, 0, -1, 0, 0, 0, -1, -2, -1, 0, -1, -2, 16, -2, -1, 0, -1, -2, -1, 0, 0, 0, -1, 0, 0]).reshape(5, 5)
    h, w = image.shape
    filtered_image = np.zeros((h, w))
    # filtered_image = [cv2.filter2D(image, -1, kernel)]
    for row_index in range(2, h - 2):
        for col_index in range(2, w - 2): 
            summ = 0
            for kernel_row in range(kernel_size):
                for kernel_col in range(kernel_size):
                    summ += image[row_index - 2 + kernel_row][col_index - 2 + kernel_col] * kernel[kernel_row][kernel_col]
            filtered_image[row_index][col_index] = summ
    return filtered_image

def linear_filter(image: np.ndarray) -> np.ndarray:
    """
    Линейный фильтры повышения резкости, возвращает отфильтрованное изоражение
    Параметры:
        image (np.ndarray): изоражение, получаемое на вход для обработки
    В фильтре заданный различные свертки, можно написать свою, главное чтобы сумма всех элиментов = 1
    """
    # kernel = np.array([[-1,-1,-1], [-1,16,-1], [-1,-1,-1]]) / 8
    kernel = np.array([[-2,-1,-2], [-1,22,-1], [-2,-1,-2]]) / 10
    # kernel = 0.2 * np.array([0, -1, -1, -1, 0, -1, 1, 1, 1, -1, -1, 1, 9, 1, -1, -1, 1, 1, 1, -1, 0, -1, -1, -1, 0]).reshape(5, 5)
    # kernel = np.array([0, -1, 0, -1, 10, -1, 0, -1, 0]).reshape(3, 3) / 6
    filtered_image = cv2.filter2D(image, -1, kernel)
    return filtered_image

def zero_crossing(image: np.ndarray) -> np.ndarray:
    """
    Для обнаружения границы как бинарного изображения широко
    используется алгоритм поиска пересечения нуля или нулевого уровня
    (zero-crossing).
    Параметры:
        image (np.ndarray): изоражение, получаемое на вход для обработки, отфильтрованное log фильтром. Изображение должно быть в градациях серого!
    """

    h, w = image.shape

    """
    T - автоматический порог для zero-crossing часто так определяют.
    h, w - высота и ширина изображения.
    """

    T = (3 / (4 * w * h)) * np.sum(np.abs(image)) * 20
    # print(T)
    """
    Важный факт - т.к. у нас имеются пороговые пикселы, необходимо определиться для них, как мы будем проводить для них проверку.
    Для этого имеется несколько вариантов:
        1) Добавить граничные пикселы с нулевым значением;
        2) Добавить граничные пикселы со значениями, идентичными анализируемому пикселу;
        3) Не обрабатывать граничные пикселы.
    Граничные пикселы используются только при обработке, в обработанное изображение они не добавляются.
    Мы будем использовать 2й вариант.
    """
    filtered_image = 255 * np.ones((h, w))
    for row_index in range(h):
        for col_index in range(w): 
            """Сравнение с верхним пикселом"""
            try:
                diff_calculation = abs(image[row_index][col_index] - image[row_index + 1][col_index])
                if (diff_calculation >= T) and \
                    (((image[row_index][col_index]) < 0 and (image[row_index + 1][col_index] > 0)) or ((image[row_index][col_index]) > 0 and (image[row_index + 1][col_index] < 0))):
                    filtered_image[row_index][col_index] = 0
            except:
                pass

            """Сравнение с нижним пикселом"""
            try:
                diff_calculation = abs(image[row_index][col_index] - image[row_index - 1][col_index])
                if (diff_calculation >= T) and \
                    (((image[row_index][col_index]) < 0 and (image[row_index - 1][col_index] > 0)) or ((image[row_index][col_index]) > 0 and (image[row_index - 1][col_index] < 0))):
                    filtered_image[row_index][col_index] = 0
            except:
                pass

            """Сравнение с правым пикселом"""
            try:
                diff_calculation = abs(image[row_index][col_index] - image[row_index][col_index + 1])
                if (diff_calculation >= T) and \
                    (((image[row_index][col_index]) < 0 and (image[row_index][col_index + 1] > 0)) or ((image[row_index][col_index]) > 0 and (image[row_index][col_index + 1] < 0))):
                    filtered_image[row_index][col_index] = 0
            except:
                pass

            """Сравнение с левым пикселом"""
            try:
                diff_calculation = abs(image[row_index][col_index] - image[row_index][col_index - 1])
                if (diff_calculation >= T) and \
                (((image[row_index][col_index]) < 0 and (image[row_index][col_index - 1] > 0)) or ((image[row_index][col_index]) > 0 and (image[row_index][col_index - 1] < 0))):
                    filtered_image[row_index][col_index] = 0
            except:
                pass
    return filtered_image

def kmeans_clustering(image: np.ndarray) -> np.ndarray:
    """
    Кластеризация изображения на 2 класса: над фронтом и пож фронтом.
    Параметры:
        image (np.ndarray): изоражение, получаемое на вход для обработки
    Возвращает массив, где ненулевая часть является той, что над фронтом(но на самом деле разницы нет, т.к. нас все равно интересует только граница)
    """
    n_clusters = 2 # т.к. нужно отделить брюки от питжака + фон, то количество кластеров = 3

    KMEANS = KMeans(n_clusters=n_clusters)
    result = KMEANS.fit(image.reshape(-1, 1))

    images = []
    for i in np.unique(result.labels_):
        image_result = np.zeros_like(image)
        # Маски для каждого элемента
        mask = (result.labels_ == i).reshape(image.shape[:])
        image_result[mask] = image[mask]
        images.append(image_result)

    if np.mean(images[0]) == max(np.mean(images[0]), np.mean(images[1])):
        return images[0]
    else:
        return images[1]
    

def create_video_from_images(images_dir: str, curr_output_dir: str = None, file_name: str = 'output_video'):
    """
    Функция ,которая объединяет фотографии в видиофайл с расширением avi.
    Параметры:
        images_dir (str): директория, из которой используются файлы для создания видиофайла;
        curr_output_dir (str): директория, в котору будет запиисан итоговый файл, по умолчанию запись идет в ту же директорию, в котором находится файл программы;
        file_name: (str): название выходного файла, по умолчанию называется output_video.
    """
    # Задаем пути к директориям
    output_file = file_name + '.avi'
    if curr_output_dir is not None:
        output_file = os.path.join(curr_output_dir, file_name)
        # Проверяем существование директорий
        if not os.path.exists(curr_output_dir):
            print(f"{curr_output_dir} does not exist, directory created.")
            os.makedirs(curr_output_dir)
    else: 
        pass

    # # Проверяем существование директорий
    if not os.path.exists(images_dir):
        raise FileNotFoundError(f"Directory with images '{images_dir}' not found.")
    
    image_format = re.findall('\.[a-z]+', os.listdir(images_dir)[0])[0][1:] # вычисляем расширение изображений
    
    # Формируем команду для ffmpeg
    ffmpeg_command = [
        'ffmpeg',
        '-framerate', '25',  # Частота кадров
        '-i', os.path.join(images_dir, 'image_%d.' + image_format),  # Шаблон имени файлов
        '-c:v', 'rawvideo',  # Кодек видео для несжатого формата
        '-pix_fmt', 'yuv420p',  # Формат пикселей для совместимости
        output_file
    ]

    # Выполняем команду ffmpeg
    try:
        subprocess.run(ffmpeg_command, check=True)
        print(f"Видео успешно создано")
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при выполнении команды ffmpeg: {e}")
    except Exception as e:
        print(f"Произошла ошибка: {e}")



def read_video_file(video_name: str, output_direcoty: str, input_directory: str = None, image_format: str = 'jpg'):
    """
    Чтение видеофайла и запись каждого кадра как изображения.
    Параметры:
        video_name (str): название видеофайла из которого читаются кадры;
        input_directory (str): название директории из которой происходит чтение файла, по умолчанию читает из той же, где находится программа;
        output_direcoty (str): название директории в которую идет запись кадров;
        image_format (str): формат, в котором буду записаны изображения, по умолчанию jpg.
    """
    if input_directory is not None:
        full_name = input_directory + '/' + video_name
    else:
        full_name = video_name
    
    if not os.path.exists(output_direcoty):
        print('Directory ' + output_direcoty + ' created')
        os.makedirs(output_direcoty)
        

    vidcap = cv2.VideoCapture(full_name)
    success, image = vidcap.read()
    count = 1
    while success:
        cv2.imwrite(output_direcoty+ f"/image_{count}." + image_format, image)    
        success, image = vidcap.read()
        if count % 100 == 0:
            print('Saved image ', count)
        count += 1
    print('All images saved')


def write_front_image(output_direcoty: str, source_directory: str, filter_on: bool = False):
    """
    Выделение фронта с предфильтрацией изображения.
    Параметры:
        output_direcoty (str): название директории в которую идет запись отфильтрованных изображений;
        source_directory (str): название директории, откуда подгружаются изображения;
        filter_on (bool): использовать предворительно фильтрацию перед выделением фронта, по умолчанию выключена.
    """

    if not os.path.exists(output_direcoty):
        print('Directory ' + output_direcoty + ' created')
        os.makedirs(output_direcoty)
        
    numbers_of_images = np.sort(np.array([re.findall('[0-9]+', im)[0] for im in sorted(os.listdir(source_directory))], dtype=int))
    image_format = re.findall('\.[a-z]+', os.listdir(source_directory)[0])[0][1:] # вычисляем расширение изображений

    for im in numbers_of_images:
        if im % 100 == 0:
            print(f'{im} images computed')
        source_path = source_directory + '/' + f'image_{im}.' + image_format
        current_image = cv2.imread(source_path, cv2.IMREAD_GRAYSCALE)
        
        if filter_on:
            # Предфильтрация
            current_image = cv2.bilateralFilter(current_image, 15, 50, 50)    
            current_image = linear_filter(current_image)
            current_image = linear_filter(current_image)
            current_image = linear_filter(current_image)
            current_image = median_filter(current_image, size=20) 
            
        current_image = kmeans_clustering(current_image)
        current_image = log_filter(current_image)
        current_image = zero_crossing(current_image)
        final_path = output_direcoty + '/' + f'image_{im}.' + image_format
        cv2.imwrite(final_path, current_image) 
    print('Computation complete')




# Основная часть

In [ ]:
"""
Модуль для записи изоображений из видио файлов
"""
read_video_file(video_name='0.715_5.10.avi', output_direcoty='video_data')

Directory video_data created
Saved image  100
Saved image  200
Saved image  300
Saved image  400
Saved image  500
Saved image  600
Saved image  700
Saved image  800
Saved image  900
Saved image  1000
All images saved


In [48]:
"""С фильтрацией изображений"""
write_front_image(output_direcoty='front_filtered_bilateral_3linear_median', source_directory='video_data', filter_on=True)

Directory front_filtered_bilateral_3linear_median created
100 images computed
200 images computed
300 images computed
400 images computed
500 images computed
600 images computed
700 images computed
800 images computed
900 images computed
1000 images computed
Computation complete


In [52]:
"""Без фильтрации изображений"""
write_front_image(output_direcoty='front', source_directory='video_data')

Directory front created
100 images computed
200 images computed
300 images computed
400 images computed
500 images computed
600 images computed
700 images computed
800 images computed
900 images computed
1000 images computed
Computation complete


In [53]:
"""Формирование из изображений видиофайла"""
create_video_from_images(images_dir='front_filtered_bilateral_3linear_median', file_name='output_video_new')

Видео успешно создано:
